In [7]:
import os
import csv
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torchmetrics import AUROC, F1Score
from transformers import BertTokenizer, BertModel, BertConfig
from torch.cuda.amp import GradScaler, autocast
import time
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from tqdm.auto import tqdm
import pandas as pd

In [8]:
class BertDataset(Dataset):
    def __init__(self, data_dir, mode, max_length=512):
        super(BertDataset, self).__init__()
        assert mode in ['train', 'val', 'test']
        self.data = pd.read_csv(os.path.join(data_dir, f'{mode}_x.csv'), index_col=0)

        if mode != 'test':
            self.labels = pd.read_csv(os.path.join(data_dir, f'{mode}_y.csv'))

        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx, 0])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']

        if hasattr(self, 'labels'):
            label = self.labels.iloc[idx].tolist()  # Assuming labels are in separate columns per class
            label = torch.tensor(label, dtype=torch.long)
            return {
                'input_ids': torch.tensor(input_ids, dtype=torch.long),
                'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
                'labels': label
            }
        else:
            return {
                'input_ids': torch.tensor(input_ids, dtype=torch.long),
                'attention_mask': torch.tensor(attention_mask, dtype=torch.long)
            }


In [9]:
model_BERT_trained = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=17)
# Load the state dictionary
state_dict = torch.load('checkpoint_1706230483.6858711.pt', map_location=torch.device('cpu'))

# Load the state dictionary into your model
model_BERT_trained.load_state_dict(state_dict)

# Put the model in evaluation mode
model_BERT_trained.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
data_dir = 'kaggle_data'
test_dataset = BertDataset(data_dir, 'test')
test_dataloader = DataLoader(test_dataset, batch_size = 20, shuffle = False)
def pred_BERT_model(model, dataloader):
    # Passer le modèle en mode évaluation
    model.eval()

    all_predictions = []
    all_indices = []

    # Pas besoin de calculer les gradients pour l'inférence
    with torch.no_grad():
        for batch in tqdm(dataloader, leave=False):
            input_ids, attention_mask = batch['input_ids'], batch['attention_mask']

            # Faire des prédictions
            outputs = model(input_ids, attention_mask=attention_mask)

            # Collecter les logits et convertir en probabilités si nécessaire
            pred = torch.sigmoid(outputs.logits).detach().cpu().tolist()
            all_predictions.extend(pred)
            # Gérer les indices pour chaque prédiction
            if 'index' in batch:
                all_indices.extend(batch['index'].tolist())  # Directement convertir si 'index' est présent
            else:
                all_indices.extend(list(range(len(pred))))  # Utiliser range converti en liste sinon

    # Créer un DataFrame pour les prédictions
    pred_df = pd.DataFrame(all_predictions, index=all_indices)

    # La fonction retourne maintenant uniquement le DataFrame des prédictions
    return pred_df



pred_df, label_df = pred_BERT_model(model_BERT_trained, test_dataloader )


  0%|          | 0/6690 [00:00<?, ?it/s]